# FalkorDB & LLamaIndex Retail Agentic Chatbot

In [1]:
%pip install llama-index-llms-openai
%pip install llama-index-graph-stores-falkordb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 7.0 MB/s eta 0:00:00
  Created wheel for falkordb: filename=falkordb-1.0.10-py3-none-any.whl size=32578 sha256=864ba8852b54c20f3337f4524176a169aaa993ade55cee73edb50dbc2df7943f
  Stored in directory: /root/.cache/pip/wheels/2d/fb/6f/1de013d3b09b753ee2e38249e14071b6e2b5e6953d88b225f0
Successfully built falkordb


In [ ]:
# My OpenAI Key
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

### Start FalkorDB

To follow every step of this tutorial, launch the image as follows:

```bash
docker run -p 6379:6379 -it --rm falkordb/falkordb:edge
```

In [ ]:
from llama_index.graph_stores.falkordb import FalkorDBGraphStore

graph_store = FalkorDBGraphStore(
    "redis://localhost:6379", decode_responses=True
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


#### Building the Knowledge Graph

In [ ]:
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from IPython.display import Markdown, display

In [ ]:
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex

# Load retail dataset
documents = SimpleDirectoryReader("sample_dataset.csv").load_data()

# Confirm data is loaded
print(f"Loaded {len(documents)} documents from the retail dataset!")

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# Configure LLM for chatbot functionality
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.llm = llm
Settings.chunk_size = 512  # Process data in chunks for better performance

In [ ]:
from llama_index.core import StorageContext

# Use FalkorDB as the backend for storing the knowledge graph
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [ ]:
# Build the knowledge graph
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,  # Extract up to 2 relationships per chunk
    storage_context=storage_context,
)

print("Knowledge graph built from the retail dataset!")

In [ ]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

# Example Query
response = query_engine.query("What are the most purchased product categories?")
print(response)

In [ ]:
query_engine = index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)

# Example Query
response = query_engine.query("Which merchants do customers buy electronics from?")
print(response)

#### Visualizing the Graph

In [2]:
%pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.4 MB/s eta 0:00:00


In [ ]:
## create graph
from pyvis.network import Network

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("falkordbgraph_draw.html")